In [ ]:
!git clone https://github.com/tanmaya48/Fake_History_Generator_NLP

Cloning into 'Fake_History_Generator_NLP'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 3), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [43]:
!ls

Fake_History_Generator_NLP  sample_data


In [ ]:
file = 'Fake_History_Generator_NLP/history.txt'


with open(file) as f:
    text = f.read()

In [ ]:
text = text.replace('\n',' lxlxl ')

for i in range(10):
    text = text.replace(str(i),'x')


text

'xxxx BC: Waun Mawn is built in West Wales lxlxl xxxx BC: Sumerian cuneiform writing system[x] and Egyptian hieroglyphs lxlxl xxxx BC: Newgrange built in Ireland lxlxl xxxx BC: Cycladic culture in Greece lxlxl xxxx BC: Norte Chico civilization begins in Peru lxlxl xxxx BC: Rise of Proto-Elamite Civilization in Iran lxlxl xxxx BC: First Dynasty of Egypt lxlxl xxxx BC: Skara Brae in Scotland lxlxl xxxx BC: Stonehenge construction begins. In its first version, it consisted of a circular ditch and bank, with xx wooden posts.[x] lxlxl xxxx BC: Cucuteni-Trypillian culture in Romania and Ukraine lxlxl xxxx BC: Jiroft civilization begins in Iran lxlxl xxxx BC: First known use of papyrus by Egyptians lxlxl xxxx-xxxx BC: Earliest evidence of autochthonous Iron production in West Africa.[x] lxlxl xxxx-xxxx BC: The East African Pastoral neolithic culture builds East Africa\'s earliest and largest monumental cemetery at Lothagam North Pillar Site. [x] lxlxl xxxx BC: Kot Diji phase of the Indus Vall

In [ ]:
import re
from bs4 import BeautifulSoup

#creating a function to clean the text
def formated( raw_review ):
    
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    

    return( " ".join(words )) 

In [ ]:
text = formated(text)

In [ ]:
len(text)

288556

In [ ]:
twos = ['in','it','is','to','of','or','on','if','an','up','bc','by','at','a']

total_words = text.split(' ')
words = set(text.split(' '))

vocab = []

for word in words:
    
    if total_words.count(word) > 3: 
        if (len(word) > 2) or (word in twos):
            vocab.append(word)

len(vocab)                

1443

In [ ]:
vocab = sorted(vocab)

In [ ]:
i = 0
end = len(text)
size = 60
skip = 3
sentences = []

while(i<end):
    
    if(i+size<=end):
        sentences.append(text[i:i+size])
    i = i+skip
        
    
len(sentences)    

96166

In [ ]:
import numpy as np
import random

In [ ]:
sequences = []

for sent in sentences:
    
    words = sent.split(' ')
    
    seq = []
    
    for word in words:
        if word in vocab:

            i = vocab.index(word)+1
            seq.append(i)
        else:
            random.seed(i)
            i = random.randint(1,len(vocab))
            seq.append(i)


    
    seq = np.array(seq)
    sequences.append(seq)
 
sequences = np.array(sequences)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
seq_len = max(len(x) for x in sequences)
seq_len

17

In [ ]:
new_sequences = []
for seq in sequences:
    
    while(len(seq)<seq_len):        
        seq = np.insert(seq,0,0)
    
    new_sequences.append(seq)
    
sequences = np.array(new_sequences)   
sequences.shape

(96166, 17)

In [ ]:
X = sequences[:,:-1]

Y = sequences[:,-1]

In [ ]:
vocab_size = len(vocab) + 1 # +1 for padding

In [ ]:
from tensorflow.keras.utils import to_categorical

# one-hot encoding the labels according to the vocab size

# The matrix is square matrix of the size of vocab_size. Each row will denote a label and it will have 
# a single +ve value(i.e 1) for that label and other values will be zero. 

y = to_categorical(Y, num_classes=vocab_size)

In [44]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential


model = Sequential()
model.add(Embedding(vocab_size, 256, input_length=seq_len-1)) # -1 since last word is ommited
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(512, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model.fit(X,y,epochs=30)

Epoch 1/30
3006/3006 [==============================] - 66s 21ms/step - loss: 5.4720 - accuracy: 0.1118
Epoch 2/30
3006/3006 [==============================] - 62s 21ms/step - loss: 3.5578 - accuracy: 0.3879
Epoch 3/30
3006/3006 [==============================] - 63s 21ms/step - loss: 3.0995 - accuracy: 0.4442
Epoch 4/30
3006/3006 [==============================] - 62s 21ms/step - loss: 2.8516 - accuracy: 0.4759
Epoch 5/30
3006/3006 [==============================] - 62s 21ms/step - loss: 2.5990 - accuracy: 0.5193
Epoch 6/30
3006/3006 [==============================] - 62s 21ms/step - loss: 2.2891 - accuracy: 0.5787
Epoch 7/30
3006/3006 [==============================] - 62s 21ms/step - loss: 1.9970 - accuracy: 0.6415
Epoch 8/30
3006/3006 [==============================] - 61s 20ms/step - loss: 1.8432 - accuracy: 0.6722
Epoch 9/30
3006/3006 [==============================] - 60s 20ms/step - loss: 1.6695 - accuracy: 0.7085
Epoch 10/30
3006/3006 [==============================] - 62s 21m

In [46]:
def text_to_input(sent):
    
    words = sent.split(' ')
    
    seq = []
    input_len = seq_len -1 # since input has the last word missing, we use seq_len - 1

    over_tex = input_len - 2
    
    for word in words:
        if word in vocab:
            seq.append(vocab.index(word)+1) # 0th index will be given to sentence padding
    
    if (len(seq) > over_tex):
        seq = seq[-over_tex:0] 
    seq = np.array(seq)
    
    while(len(seq)<input_len):        
        seq = np.insert(seq,0,0)
        
        
    
    seq = np.array([seq])
    return seq   

In [47]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [49]:
for n in range(10):
    input_text = 'lxlxl'
    prev_word = ''
    for i in range(40):
        
        seed = text_to_input(input_text)
            
        pred = sample(model.predict(seed)[0],temperature=2)   
        
        word = vocab[pred-1]

        if word == 'lxlxl':
            break
          
        if word != 'des':  # for this specific model
          input_text = input_text +' '+word

    print('-----------------------------------------')
    print(input_text[6:])  
    print('-----------------------------------------')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


-----------------------------------------
xxx castile and norway off several georgia a first expedition red sri lanka and poland poland in after poland in poland a around is in poland in anatolia with poland poland to an poland poland of and at name poland
-----------------------------------------
-----------------------------------------
domination yemen by malacca into and establishes of imperial leadership old order navarre all a georgia poland
-----------------------------------------
-----------------------------------------
the new ternate to to chang all macau in the world more
-----------------------------------------
-----------------------------------------
domination civil war end indo pius khrushchev
-----------------------------------------
-----------------------------------------
xxx castile dies harold height raids wars saudi half defeat a close july league name an becomes georgia at in the poland georgia an to first becomes and poland iii of under set a set anatolia
--